<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-13 14:49:58
-------------------
qualified stocks: 93
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  2.51 L
Current:  1.33 C
-------------------
Today PnL: 62.98 K (0.48%)
Current PnL: -21.78 L (-14.81%)
CY Booked + Current PnL: -13.51 L (-9.19%)
-------------------
Total profit:  2.50 L
Total loss:  -24.28 L
-------------------
Total Booked + Current PnL: 13.47 L (11.22%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.21%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 81.25 L (61.05%)
Deployed:  1.20 C
Current:  1.33 C
CAGR/XIRR %: 6.63%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,756.01,-18.25,H-LC,6.03,112256.0,7944.0,8711.0,5.18,7.62,7.76,15.96,11.0,0.91,0.86,25.80,X5K,ATH,METALS
68,SBIN,760.30,863.00,-12.30,M-LC,3.85,218002.0,16522.0,10682.0,0.25,8.20,4.90,13.51,61.0,1.55,1.67,21.83,XY25,NTT,BANKS
55,MEDANTA,1087.93,1486.00,10.86,H-MC,13.95,158340.0,32140.0,14029.0,-1.11,25.47,8.86,36.59,111.0,2.29,1.21,41.17,XY24,NTT,HEALTHCARE
83,TTKPRESTIG,769.29,770.00,93.17,M-SC,10.53,83244.0,-17533.0,17623.0,0.39,-17.40,21.17,0.09,243.0,-0.99,0.64,9.36,OX40N,NTT,DURABLES
51,LALPATHLAB,2838.53,3545.00,-25.70,H-SC,9.59,222142.0,29122.0,18926.0,0.33,15.09,8.52,24.89,134.0,1.54,1.70,41.96,X40N,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DABUR,505.20,735.00,-9.78,H-MC,3.75,196384.0,-644.0,90258.0,0.13,-0.33,45.96,45.49,103.0,-0.01,1.50,12.67,XY24,BTT,FMCG
90,VOLTAS,1278.28,1918.49,-8.49,H-MC,7.66,192015.0,273.0,95758.0,2.82,0.14,49.87,50.08,101.0,0.00,1.47,6.95,XY25,ATH,AC
56,NATIONALUM,189.63,247.44,-49.46,H-MC,2.04,100907.0,403.0,30232.0,2.08,0.40,29.96,30.49,80.0,0.01,0.77,32.94,MH,ATH,METALS
92,WIPRO,243.46,420.00,-14.70,M-LC,7.15,150090.0,-855.0,110316.0,0.15,-0.57,73.50,72.51,54.0,-0.01,1.15,5.42,XR,NTT,IT


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-20.12,H-LC,2.31,213514.0,9014.0,32390.0,2.42,4.41,15.17,20.25,10.0,0.28,1.64,13.53,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-12.80,H-LC,2.84,170102.0,-31284.0,118901.0,0.94,-15.53,69.90,43.51,27.0,-0.26,1.30,18.41,X40,BTT,PAINTS
57,NESTLEIND,2268.60,2755.0,-16.37,H-LC,3.61,128723.0,-136703.0,193612.0,0.83,-51.50,150.41,21.44,8.0,-0.71,0.99,3.87,XY25,NTT,FMCG
53,LTIM,5564.16,7230.2,-7.23,H-LC,3.76,183996.0,-16314.0,76285.0,0.08,-8.14,41.46,29.94,17.0,-0.21,1.41,27.11,X200,ATH,IT
4,APOLLOHOSP,7098.95,8285.0,-20.28,H-LC,5.41,159203.0,3026.0,23069.0,-0.31,1.94,14.49,16.71,19.0,0.13,1.22,19.56,X40N,BTT,HEALTHCARE


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.88,72576.0,-40973.0,81024.0,5.73,-36.08,111.64,35.27,268.0,-0.51,0.56,77.63,XR,NTT,HOTELS
36,HINDALCO,651.95,756.01,-18.25,H-LC,6.03,112256.0,7944.0,8711.0,5.18,7.62,7.76,15.96,11.0,0.91,0.86,25.80,X5K,ATH,METALS
88,VALIANTORG,512.64,838.00,-314.60,H-SC,17.24,91260.0,-42026.0,126623.0,4.73,-31.53,138.75,63.47,143.0,-0.33,0.70,52.97,XR,NTT,CHEMICALS
75,SURYODAY,189.78,240.00,42.54,H-SC,4.62,100709.0,-49217.0,88886.0,3.93,-32.83,88.26,26.46,137.0,-0.55,0.77,29.02,XR,NTT,BANKS
66,SAMMAANCAP,170.35,326.00,-163.64,M-SC,4.90,73728.0,-28482.0,121872.0,2.85,-27.87,165.30,91.37,206.0,-0.23,0.56,19.68,XY25,NTT,FINANCE


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
10,BAJAJHFL,122.26,152.0,-15.33,H-MC,4.91,178235.0,-22271.0,71044.0,-3.32,-11.11,39.86,24.33,98.0,-0.31,1.36,2.78,X40N,BTT,FINANCE
72,SIS,477.00,528.0,2137.31,H-SC,6.79,60197.0,-20893.0,29563.0,-2.71,-25.77,49.11,10.69,155.0,-0.71,0.46,22.00,OX40N,NTT,MISC
13,BATAINDIA,1550.24,2096.0,-38.58,M-SC,6.56,91790.0,-36880.0,82180.0,-2.63,-28.66,89.53,35.20,209.0,-0.45,0.70,0.00,X40,NTT,FOOTWEAR
59,QUESS,296.83,986.0,-41.05,M-SC,42.04,58802.0,-6204.0,157137.0,-2.50,-9.54,267.23,232.18,198.0,-0.04,0.45,0.49,XY24,NTT,MISC
29,EASEMYTRIP,18.12,26.4,-10.04,M-SC,24.18,86749.0,-90066.0,170861.0,-2.31,-50.94,196.96,45.70,197.0,-0.53,0.66,0.91,XY24,NTT,TRAVEL


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,93.17,M-SC,10.53,83244.0,-17533.0,17623.0,0.39,-17.40,21.17,0.09,243.0,-0.99,0.64,9.36,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,83.20,H-SC,20.55,115695.0,-26784.0,46301.0,-0.74,-18.80,40.02,13.70,163.0,-0.58,0.89,36.55,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-19.81,H-SC,3.10,114408.0,-29395.0,56140.0,-0.50,-20.44,49.07,18.60,148.0,-0.52,0.88,26.25,OX40N,NTT,CERAMICS
49,KANSAINER,299.63,340.0,-69.07,H-SC,2.52,213489.0,-56178.0,92505.0,-0.48,-20.83,43.33,13.47,142.0,-0.61,1.63,7.98,XY24,NTT,PAINTS
43,INDIGOPNTS,1407.73,1408.0,108.15,M-SC,3.12,136363.0,-38196.0,38223.0,0.05,-21.88,28.03,0.02,221.0,-1.00,1.04,18.25,OX40N,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-28.36,H-LC,15.84,236964.0,-58970.0,152889.0,0.09,-19.93,64.52,31.73,2.0,-0.39,1.81,1.17,X40,BTT,IT
45,INFY,1461.46,2275.00,-21.75,H-LC,10.62,255809.0,-5792.0,151413.0,0.35,-2.21,59.19,55.67,4.0,-0.04,1.96,3.76,X40,BTT,IT
89,VBL,492.64,671.64,-8.75,H-LC,10.42,276936.0,11896.0,84410.0,0.10,4.49,30.48,36.33,5.0,0.14,2.12,18.39,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.62,H-LC,10.16,198178.0,-18341.0,111237.0,0.20,-8.47,56.13,42.91,7.0,-0.16,1.52,3.79,AR,NTT,ELECTRICAL
57,NESTLEIND,2268.60,2755.00,-16.37,H-LC,3.61,128723.0,-136703.0,193612.0,0.83,-51.50,150.41,21.44,8.0,-0.71,0.99,3.87,XY25,NTT,FMCG


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,10.86,H-MC,13.95,158340.0,32140.0,14029.0,-1.11,25.47,8.86,36.59,111.0,2.29,1.21,41.17,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,10.86,H-MC,13.95,158340.0,32140.0,14029.0,-1.11,25.47,8.86,36.59,111.0,2.29,1.21,41.17,XY24,NTT,HEALTHCARE
50,KPIGREEN,497.21,731.35,9.02,H-SC,12.67,130082.0,4785.0,54218.0,0.26,3.82,41.68,47.09,141.0,0.09,1.00,62.81,MH,ATH,POWER
15,BLUESTARCO,1646.70,2326.38,2.13,H-MC,7.15,176230.0,11560.0,56411.0,0.96,7.02,32.01,41.28,91.0,0.20,1.35,15.56,X40N,ATH,AC
19,CAMS,3643.00,5211.76,-5.83,H-SC,3.47,107184.0,5180.0,38747.0,1.57,5.08,36.15,43.06,123.0,0.13,0.82,23.06,X40N,ATH,MISC
90,VOLTAS,1278.28,1918.49,-8.49,H-MC,7.66,192015.0,273.0,95758.0,2.82,0.14,49.87,50.08,101.0,0.00,1.47,6.95,XY25,ATH,AC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-9.65,H-MC,7.41,167164.0,16251.0,41055.0,2.82,10.77,24.56,37.97,92.0,0.40,1.28,38.80,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-51.65,H-SC,12.19,134933.0,11597.0,122168.0,0.98,9.40,90.54,108.45,119.0,0.09,1.03,33.86,AR,ATH,MISC
50,KPIGREEN,497.21,731.35,9.02,H-SC,12.67,130082.0,4785.0,54218.0,0.26,3.82,41.68,47.09,141.0,0.09,1.00,62.81,MH,ATH,POWER
91,WHIRLPOOL,1219.98,2270.00,-44.70,M-SC,4.02,93375.0,1876.0,76876.0,0.15,2.05,82.33,86.07,219.0,0.02,0.72,35.08,XR,NTT,DURABLES
56,NATIONALUM,189.63,247.44,-49.46,H-MC,2.04,100907.0,403.0,30232.0,2.08,0.40,29.96,30.49,80.0,0.01,0.77,32.94,MH,ATH,METALS


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.88,72576.0,-40973.0,81024.0,5.73,-36.08,111.64,35.27,268.0,-0.51,0.56,77.63,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-2.99,H-SC,16.25,90257.0,-10706.0,105240.0,1.24,-10.60,116.60,93.63,149.0,-0.10,0.69,34.44,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-18.70,H-SC,14.98,92370.0,-5610.0,27480.0,0.95,-5.73,29.75,22.32,152.0,-0.20,0.71,33.97,XR,ATH,FINANCE
36,HINDALCO,651.95,756.01,-18.25,H-LC,6.03,112256.0,7944.0,8711.0,5.18,7.62,7.76,15.96,11.0,0.91,0.86,25.80,X5K,ATH,METALS
84,UJJIVANSFB,52.77,60.00,83.20,H-SC,20.55,115695.0,-26784.0,46301.0,-0.74,-18.80,40.02,13.70,163.0,-0.58,0.89,36.55,OX40N,NTT,BANKS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-9.65,H-MC,7.41,167164.0,16251.0,41055.0,2.82,10.77,24.56,37.97,92.0,0.40,1.28,38.80,AR,ATH,AUTO
70,SHALBY,261.39,327.00,966.99,M-SC,7.10,145080.0,-36063.0,81535.0,1.63,-19.91,56.20,25.10,232.0,-0.44,1.11,16.27,XY24,NTT,HEALTHCARE
51,LALPATHLAB,2838.53,3545.00,-25.70,H-SC,9.59,222142.0,29122.0,18926.0,0.33,15.09,8.52,24.89,134.0,1.54,1.70,41.96,X40N,NTT,HEALTHCARE
89,VBL,492.64,671.64,-8.75,H-LC,10.42,276936.0,11896.0,84410.0,0.10,4.49,30.48,36.33,5.0,0.14,2.12,18.39,X40N,ATH,FMCG
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.88,72576.0,-40973.0,81024.0,5.73,-36.08,111.64,35.27,268.0,-0.51,0.56,77.63,XR,NTT,HOTELS


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.44
1,25,41.21
2,50,70.91


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.48
LC    34.28
MC    26.27
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.95
X40N     15.40
X40      13.34
XY25     10.70
XR        9.06
AR        8.43
OX40N     7.53
X5K       2.30
MH        1.77
X200      1.41
SR        1.14
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    25.91
H-SC    24.57
H-MC    23.31
M-SC    13.40
M-LC     7.28
M-MC     2.62
L-SC     1.51
L-LC     1.09
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.27,-10.71,46.29
IT,11.97,-19.50,79.19
FINANCE,8.28,-20.61,65.24
BANKS,7.66,-13.98,62.37
MISC,5.87,-19.93,88.68
PAINTS,5.63,-18.39,41.72
HEALTHCARE,5.24,4.12,20.09
ELECTRICAL,4.72,-14.06,63.45
AUTO,4.49,-15.02,65.06


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2855020.0,23
XR,1112387.0,13
AR,1107393.0,10
X40,831094.0,10
XY25,651323.0,8
X40N,597059.0,12
OX40N,476662.0,10
SR,194830.0,2
X5K,138837.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2809839.0,28
M-SC,1495067.0,17
H-LC,1355738.0,18
H-MC,1333066.0,18
M-LC,427623.0,5
M-MC,335982.0,2
L-SC,265728.0,3
L-MC,58466.0,1
L-LC,43831.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      938570.0      6
M-SC       XY24      871244.0      7
H-SC       AR        673980.0      5
           XR        524147.0      6
H-LC       X40       518074.0      5
H-MC       XY24      493799.0      5
M-MC       XY24      335982.0      2
H-LC       XY25      254694.0      3
           AR        231696.0      2
H-MC       X40       230840.0      4
H-LC       X40N      227352.0      5
H-MC       XY25      220244.0      2
H-SC       X40N      218521.0      4
           OX40N     205573.0      4
           SR        194830.0      2
M-LC       XY24      176499.0      2
L-SC       XR        176193.0      2
M-SC       AR        160662.0      2
H-MC       X40N      151186.0      3
M-SC       OX40N     133200.0      4
M-LC       X5K       130126.0      1
M-SC       XR        125909.0      2
           XY25      121872.0      1
H-MC       XR        117356.0      1
M-LC       XR        110316.0      1
L-SC       OX40N      89535.0      1
M-SC       X40        82180.0      1
H-LC       X200       76285.0      1
L-MC       XR         58466.0      1
H-SC       MH         54218.0      1
H-MC       OX40N      48354.0      1
L-LC       XY25       43831.0      1
H-MC       AR         41055.0      1
H-LC       XY24       38926.0      1
H-MC       MH         30232.0      1
M-LC       XY25       10682.0      1
H-LC       X5K         8711.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
